In [1]:
# Install required libraries
!pip install transformers

# Import necessary modules
from transformers import pipeline
import concurrent.futures

# Pick model
model_name = "facebook/bart-large-cnn"

# Define summarization pipeline
summarizer = pipeline(
    "summarization",
    model=model_name,
    framework="pt"
)

In [2]:

# Example text
example_text = """
World War II, spanning from 1939 to 1945, was one of the deadliest and most significant conflicts in human history. It involved the majority of nations, divided into two opposing alliances: the Allies and the Axis powers. Here is a concise history of World War II.

The war began on September 1, 1939, when Germany, under Adolf Hitler, invaded Poland. In response, France and the United Kingdom declared war on Germany. The German military swiftly overwhelmed Poland, utilizing a tactic known as blitzkrieg. Hitler's aggression continued, leading to the conquest of Denmark, Norway, the Netherlands, Belgium, and France in 1940.

In 1941, Germany violated a non-aggression pact with the Soviet Union and launched a massive invasion. Initially, the Germans made significant gains but faced fierce resistance and harsh weather conditions. Eventually, the Soviet Union, under Joseph Stalin, repelled the German forces and began pushing them back toward Berlin.

Meanwhile, in the Pacific, Imperial Japan sought to expand its empire. In 1941, Japan attacked the United States naval base at Pearl Harbor, drawing America into the war. The Pacific theater witnessed brutal battles, such as Guadalcanal, Midway, and Iwo Jima, as the Allies gradually pushed back Japanese forces.

Back in Europe, the turning point came in 1942 with the Battle of Stalingrad. The Soviet Union successfully defended the city against a prolonged German siege, marking a major setback for Hitler's forces. In 1944, the Allies launched Operation Overlord, a massive amphibious invasion of German-occupied France. D-Day, as it is famously known, resulted in the establishment of a western front, leading to the liberation of Paris and the subsequent push into Germany.

On the Eastern Front, the Soviet Union launched a massive offensive, pushing the German forces steadily backward. In 1945, Soviet troops reached Berlin, and Hitler committed suicide. Germany surrendered unconditionally in May 1945. Meanwhile, in the Pacific, the United States utilized atomic bombs on the cities of Hiroshima and Nagasaki, forcing Japan's surrender in August 1945.

World War II resulted in the deaths of an estimated 70-85 million people, including civilians and military personnel. It also saw the implementation of the Holocaust, a systematic genocide that targeted six million Jews. The war led to the emergence of the United States and the Soviet Union as superpowers and the subsequent Cold War between them.

In an effort to prevent future global conflicts, the United Nations was established in 1945. It aimed to promote peace and international cooperation. The devastating impact of World War II and its far-reaching consequences forever changed the course of history and serves as a stark reminder of the horrors of war
"""


In [3]:
# Split the input text into chunks to fit within the model's maximum token limit
max_chunk_length = 1024  # Maximum length of each chunk
text_chunks = [example_text[i:i+max_chunk_length] for i in range(0, len(example_text), max_chunk_length)]

# Function to summarize a chunk
def summarize_chunk(chunk):
    summary = summarizer(chunk, min_length=30, max_length=100)
    return summary[0]['summary_text']

# Create a list to store the generated summaries
all_summaries = []

# Generate summaries for each text chunk using parallel processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit summarization tasks for each chunk
    future_summaries = [executor.submit(summarize_chunk, chunk) for chunk in text_chunks]

    # Retrieve the summaries as they complete
    for future in concurrent.futures.as_completed(future_summaries):
        summary = future.result()
        all_summaries.append(summary)

# Combine the summaries into a single coherent summary
combined_summary = " ".join(all_summaries)

# Print the summarized text
print(combined_summary)

World War II, spanning from 1939 to 1945, was one of the deadliest and most significant conflicts in human history. It involved the majority of nations, divided into two opposing alliances: the Allies and the Axis powers. World War II resulted in the deaths of an estimated 70-85 million people. It also saw the implementation of the Holocaust, a systematic genocide that targeted six million Jews. The war led to the emergence of the United States and the Soviet Union as superpowers. In 1941, Japan attacked the United States naval base at Pearl Harbor, drawing America into the war. The Pacific theater witnessed brutal battles, such as Guadalcanal, Midway, and Iwo Jima. Back in Europe, the turning point came in 1942 with the Battle of Stalingrad.
